In [12]:
# Import required Packages

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
import time
from bs4 import BeautifulSoup
import pandas as pd

In [34]:
# Declare constants
training_source = 'AcloudGuru'

# Course names
course_associate_developer ='aws-developer-associate'
course_associate_architect ='aws-soultions-architect-associate' 

# Course urls
url_associate_developer = 'https://acloud.guru/learn/aws-certified-developer-associate'
url_associate_architect ='https://acloud.guru/learn/aws-certified-solutions-architect-associate'


In [37]:
# Set Current Course & URL 
course_name = course_associate_developer
start_url =url_associate_developer

file_name = training_source+'-'+course_name

In [38]:
# Navigate to page with course contents and grab the page
with webdriver.Firefox() as driver:
    wait = WebDriverWait(driver, 10)
    driver.get(start_url)

    time.sleep(10)
 
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
    time.sleep(3)


    # And grab the page HTML source
    html_page = driver.page_source
    
    # Save a local Copy
    with open(file_name+'.html','w') as file:
        file.write(html_page)
          

In [39]:
# Read the Saved HTML File 
source_page =''   
# Read the local saved copy
with open(file_name+'.html','r') as file:
    source_page = file.read()  

In [40]:
# Start Processing the file
# Parse the HTML with BeautifulSoup and create a soup object
soup = BeautifulSoup(source_page)

table_columns = []
table_columns.append('trainingPortal')
table_columns.append('course')
table_columns.append('title order')
table_columns.append('title')
table_columns.append('lectures')
table_columns.append('section title')
table_columns.append('duration')

table_data = []
title_order = 0

# Select course list
course_content = soup.select('div[class="curriculum-section"]') 

for ele in course_content:
    row_data = []
    title_order = title_order + 1
    no_of_lectures= ''
    heading = ele.select('h2[class="curriculum--heading"]')
    duration = heading[0].select('span')[0].text
    title = heading[0].text.replace(duration,'')
    row_data.append(training_source)
    row_data.append(course_name)
    row_data.append(title_order)
    row_data.append(title)
    row_data.append(no_of_lectures)
    row_data.append('')
    row_data.append(duration)
    table_data.append(row_data)
    sub_sections = ele.select('span[class="curriculum--metadata-title"]')
    sub_sections_count = 0
    for sub_title in sub_sections:
        sub_sections_count +=1
        sub_row = row_data.copy()
        sub_row[5] = sub_title.text
        sub_row[6] = ''
        table_data.append(sub_row)
    row_data[4] =sub_sections_count
 

In [41]:
# Create a Pandas DataFrame
df = pd.DataFrame(table_data, columns=table_columns)
df.to_excel(file_name+".xlsx") 
df

,trainingPortal,course,title order,title,lectures,section title,duration
0,AcloudGuru,aws-developer-associate,1,Chapter 1 Introduction,3,,09:24
1,AcloudGuru,aws-developer-associate,1,Chapter 1 Introduction,,Introduction,
2,AcloudGuru,aws-developer-associate,1,Chapter 1 Introduction,,What Can I Skip?,
3,AcloudGuru,aws-developer-associate,1,Chapter 1 Introduction,,Exam Blue Print,
4,AcloudGuru,aws-developer-associate,2,Chapter 2 Beginners Guide to IAM,4,,42:57
...,...,...,...,...,...,...,...
131,AcloudGuru,aws-developer-associate,12,Chapter 12 Updates Based On Student Feedback,,Chapter Summary,
132,AcloudGuru,aws-developer-associate,12,Chapter 12 Updates Based On Student Feedback,,AWS Certified Developer - Associate 2020 - Upd...,
133,AcloudGuru,aws-developer-associate,13,Chapter 13 Summary,2,,2:17:55
134,AcloudGuru,aws-developer-associate,13,Chapter 13 Summary,,Summary and next steps,
